# Neural Network in Code

## Dependency

In [10]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import gcsfs
import h5py
import pickle

## Parameter

## Regression

In [7]:
# Number of features
p = 5

model = tf.keras.Sequential(name="Demo model")
model.add(tf.keras.layers.Dense(units=120, activation='relu', kernel_initializer='he_uniform', input_shape=(p,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=120, activation='relu', kernel_initializer='he_uniform', input_shape=(p,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
# Dense(1) for regression in the output layer
model.add(tf.keras.layers.Dense(1))

model.compile(
    # If we wanna change loss function
    loss=tf.keras.losses.Huber(delta=0.1),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    # If we wanna monitor loss over training iterations
    metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.MeanAbsoluteError()
    ]
)

model.summary()

# Data preparation
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train_scaled = np.clip(X_train_scaled, -4.0, 4.0)
# X_test_scaled = np.clip(X_test_scaled, -4.0, 4.0)

# To terminate training
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=30,
    mode='min',
    restore_best_weights=True
)

# Training
# history = model.fit(
#     X_train_scaled,
#     y_train,
#     batch_size=10,
#     epochs=100,
#     callbacks=[callbacks],
#     validation_data=(X_vali_scaled, y_vali),
#     verbose=2
# )

# Save model
# model.save('nn')
# model.save('nn.h5')

# Save scaler
pickle.dump(scaler, open('scaler.pkl', 'wb'))

# Load model
# model = tf.keras.models.load_model('nn')

# Prediction
# pred = model.predict(X_test_scaled)

Model: "Demo model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 120)               720       
                                                                 
 batch_normalization_6 (Batc  (None, 120)              480       
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 120)               14520     
                                                                 
 batch_normalization_7 (Batc  (None, 120)              480       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 120)               0         
                                                                 
 dense_10 (Dense)            (None, 1)                 1

## Load TensorFlow .h5 model from Cloud Storage

In [ ]:
# Running below code requires that this script is authenticated to use the Google Cloud Project
# by GOOGLE_APPLICATIONS_CREDENTIAL environment variable or service account
fs = gcsfs.GCSFileSystem(project='GOOGLE_CLOUD_PROJECT')

# Use gsutil URL found in Cloud Storage
with fs.open('gs://PROJECT_NAME/FOLDER/MODEL.h5', 'rb') as model_file:
    model_gcs = h5py.File(model_file, 'r')
    model = tf.keras.models.load_model(model_gcs)